In [53]:
import math
import json
import numpy as np
import re
from lxml import etree
from functools import reduce
from operator import mul

# What file are the raw scrapes from LMFDB in?
filename = 'raw_lmfdb'
output_filename = 'l1_values'

In this seciton, we parse the raw pulls from LMFDB, extracting the discriminant and the size of the class group

In [24]:
with open(filename) as f:
    all_the_text = f.read()
splits = [x for x in re.split(r'=========*', all_the_text.strip()) if x.strip()]

In [25]:
def parse_disc(disc):
    """
    From a discriminant of the form r'\( [\-,] 2^{5} \cdot 3^{2} \)'
    extract the actual value of the discriminant.
    
    :return: tuple of whether the discriminant is negative, and the
             the absolute value of the discriminant
    :rtype: (bool, int)
    """
    disc = disc[2:-2]
    
    if '-' in disc:
        is_neg = True
        disc = disc.split(',')[1]
    else:
        is_neg = False
        
    factors = disc.split(r'\cdot')
    
    output = []
    for factor in factors:
        split = re.findall(r'(\d+)', factor)
        if len(split) == 1:
            output.append([int(split[0]), 1])
        else:
            output.append([int(x) for x in split])
    return is_neg, output

In [26]:
def class_group_size(cg):
    """
    From the LMFDB representation (which is either "Trivial" or as the a-invariants
    of the class group as a finite abelian group), return its order.
    
    :return: The order of the class group
    :rtype: int
    """
    if 'Trivial' in cg:
        return 1
    cg = eval(cg.strip())
    return reduce(mul, cg, 1)

In [27]:
# Actual do the parsing

answers = []
for i, split in enumerate(splits):
    root = etree.HTML(split)
    tds = root.xpath('//td')
    print("Processing {}/{}".format(i, len(splits))
    for disc, cg in zip(tds[2::5], tds[4::5]):
        disc = parse_disc(disc.text)
        cgs = class_group_size(cg.text)
        answers.append([disc, cgs])
print("Done")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In this section, we actually compute the L1 values of these fields.

In [63]:
def foo(a, b):
    return a * b[0] ** b[1]

l1s = {}
total_neg = 0

for (is_neg, disc), cgs in answers:
    if not is_neg:
        continue
    disc = reduce(foo, disc, 1)
    l1 = 2 * math.pi * cgs / 2 / math.sqrt(disc)
    if is_neg:
        disc *= -1
        total_neg += 1
    l1s[disc] = l1

In [66]:
with open(output_filename, 'w') as f:
    json.dump(l1s, f)

Finally, we verify that we haven't missed anything!

In [67]:
def sqfree_sieve(N):
    output = np.ones(N, dtype=bool)
    output[0] = output[1] = False
    for i, n in enumerate(output):
        if not n:
            continue
        i_squared = i * i
        output[i_squared::i_squared] = False
    return output

def is_fund_disc(D, sieve):
    if D == -4:
        return True
    if D in [0, 1]:
        return False
    
    if D % 4 == 0:
        D_by_four = D // 4
        if D_by_four % 4 in [2, 3] and sieve[abs(D_by_four)]:
            return True
        return False
    if D % 4 != 1:
        return False
    return sieve[abs(D)]

N = 10000000
sieve = sqfree_sieve(N)
discs = [is_fund_disc(n, sieve) for n in range(-N+1, N)]

In [68]:
found_discs = np.abs(np.array(list(l1s.keys())))
max_l1 = found_discs.max()
found = np.zeros(max_l1+1, dtype=bool)
is_disc = np.array([is_fund_disc(-D, sieve) for D in range(max_l1+1)], dtype=bool)
found[found_discs] = True

how_many_missed = np.sum(np.ediff1d(np.logical_xor(found, is_disc)))
print("We missed {} fields pulling from LMFDB".format(how_many_missed))

0

If you *did* miss any, then the following list will tell you which ones you missed and you can pull them!

In [60]:
not_found = [d for d in range(len(found)) if found[d] != is_disc[d]]

In [41]:
import math, re

with open('/Users/kevin/hold', 'r') as f:
    s = f.read()
    
splits = [re.findall(r'=(\d*): ([0-9.]*)', x)[0] for x in s.strip().split('\n')]
[float(y) / math.log(float(x)) for x, y in splits]

[0.48378386078055174,
 0.5039887049089553,
 0.514611988036161,
 0.5212943646179035,
 0.5266365353424388,
 0.5302289598223249,
 0.5338062540760946,
 0.5364273664854804,
 0.5379891496469114,
 0.5396654540878858,
 0.5414596354158635,
 0.543221752709458,
 0.5448474164588399,
 0.5460072352303548,
 0.5473343258976774,
 0.5484580521198494,
 0.5492949022262587,
 0.550270924192911,
 0.5507703522394669,
 0.5517037146001371,
 0.5523515234127141,
 0.5532064813669388,
 0.5538464247266743,
 0.5545213316512065,
 0.5552103744093145,
 0.5556437461377821,
 0.5561857037358937,
 0.556641712448417,
 0.557015470193476,
 0.5577014901621951,
 0.5582566892707881,
 0.5586978211603272,
 0.5591611251338741,
 0.5596860230788867,
 0.5600490738563586,
 0.5603476715717666,
 0.5607823905053427,
 0.5610400607107957,
 0.5614027412980148,
 0.5617812389899891,
 0.5621524784762881,
 0.5625094663637857,
 0.5627915827380486,
 0.5631424212764992,
 0.5633606253411054,
 0.5636666843936275,
 0.5639289440931702,
 0.56423814549978